In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from keras.utils import to_categorical
import tensorflow.keras.layers as k
from scipy import stats

Using TensorFlow backend.


In [7]:
# Data was generated using R so run this portion to convert RDS to pandas dataframe
pandas2ri.activate()
readRDS = robjects.r['readRDS']
data = readRDS('demand_wide.RDS')
data = pandas2ri.ri2py(data)

In [44]:
windowSize = 3
batchSize = 1
testSize = 2

In [48]:
        # Sample rows according to their scale. The larger the scale, the higher the probability
        # of sampling form that observation
        # Then randomly sample a window fixed to length windowSize / testSize
        rows = np.random.choice(range(0, data.shape[0]), size=batchSize, p=data["sampling_probability"])
        all_cols = [col for col in data if col.startswith('2')]
        start = np.random.randint(0, len(all_cols) - windowSize - testSize, size=1)
        train_cols = all_cols[int(start):int(start + windowSize)]
        test_cols = all_cols[int(start + windowSize):int(start + windowSize + testSize)]
        
        # Store number and subclass will be passed to embedding layers
        stores_train = data.iloc[rows,]["store_number"].tolist()
        biz_cd_train = data.iloc[rows,]["biz_cd_int"].tolist()
        
        # Scale factor is used to scale training data and will be passed into model
        # to be used to rescale the network output
        scale_factor = data.iloc[rows,]["scale_factor"].tolist()
        
        # We set the max number of classes to 53
        week_train = [map(int, [week[-2:] for week in train_cols])]
        week_train = np.reshape(week_train, (batchSize, windowSize, 1))
        # week_train = np.reshape(np.repeat(week_train, batchSize, axis=0), (batchSize, windowSize, 53))

        week_test = to_categorical([map(int, [week[-2:] for week in test_cols])], num_classes=53)
        week_test = np.reshape(np.repeat(week_test, batchSize, axis=0), (batchSize, testSize, 53))
        
        x_data = data.iloc[rows,][train_cols].div(data.iloc[rows,]["scale_factor"], axis = 0).values
        x_data = np.reshape(x_data, (batchSize, windowSize, 1))
        x_data = np.concatenate((x_data, week_train), axis=2)
        
        y_data = data.iloc[rows,][test_cols].values
        y_data = np.reshape(y_data, (batchSize, testSize, 1))
#         y_data = np.concatenate((y_data, week_test), axis=2)

array([[[ 0.        , 27.        ],
        [ 0.        , 28.        ],
        [ 0.68108108, 29.        ]]])

In [51]:
check = next(train_gen)

StopIteration: 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [42]:
def custom_data_gen(data, batchSize, probs, windowSize, testSize):
    while True:
        
        # Sample rows according to their scale. The larger the scale, the higher the probability
        # of sampling form that observation
        # Then randomly sample a window fixed to length windowSize / testSize
        rows = np.random.choice(range(0, data.shape[0]), size=batchSize, p=data["sampling_probability"])
        all_cols = [col for col in data if col.startswith('2')]
        start = np.random.randint(0, len(all_cols) - windowSize - testSize, size=1)
        train_cols = all_cols[int(start):int(start + windowSize)]
        test_cols = all_cols[int(start + windowSize):int(start + windowSize + testSize)]
        
        # Store number and subclass will be passed to embedding layers
        stores_train = data.iloc[rows,]["store_number"].tolist()
        biz_cd_train = data.iloc[rows,]["biz_cd_int"].tolist()
        
        # Scale factor is used to scale training data and will be passed into model
        # to be used to rescale the network output
        scale_factor = data.iloc[rows,]["scale_factor"].tolist()
        
        # We set the max number of classes to 53
        week_train = [map(int, [week[-2:] for week in train_cols])]
        # week_train = np.reshape(np.repeat(week_train, batchSize, axis=0), (batchSize, windowSize, 53))

        week_test = to_categorical([map(int, [week[-2:] for week in test_cols])], num_classes=53)
        week_test = np.reshape(np.repeat(week_test, batchSize, axis=0), (batchSize, testSize, 53))
        
        x_data = data.iloc[rows,][train_cols].div(data.iloc[rows,]["scale_factor"], axis = 0).values
        x_data = np.reshape(x_data, (batchSize, windowSize, 1))
        x_data = np.concatenate((x_data, week_train), axis=2)
        
        y_data = data.iloc[rows,][test_cols].values
        y_data = np.reshape(y_data, (batchSize, testSize, 1))
#         y_data = np.concatenate((y_data, week_test), axis=2)
        
        yield [x_data, np.array(biz_cd_train), np.array(scale_factor), np.array(stores_train)], y_data

In [5]:
def custom_loss(y_true, y_pred):
    y_pred = y_pred + 1e-7
    y_true = y_true + 1e-7

    # Need to extract input vectors and set the shapes of tensors
    mu = tf.reshape(y_pred[:,:,0], [-1])
    alpha = tf.reshape(y_pred[:,:,1], [-1])
    scale = tf.reshape(y_pred[:,:,2], [-1])
    y_true = tf.reshape(y_true[:,:,0], [-1])

    # need to rescale mu and alpha
    mu = mu * scale
    alpha = alpha / tf.sqrt(scale)
    
    # Using tf probability to calculate log loss
    loss = tfp.distributions.NegativeBinomial(mu, alpha).log_prob(y_true)

    return -tf.reduce_sum(loss, axis=-1)

In [8]:
# Set all input layers
input_demand = k.Input(shape = (windowSize, 54), name = 'demand')
input_store = k.Input(shape = (1,), name = 'store')
input_biz_cd = k.Input(shape = (1,), name = 'biz_cd')
input_scale_factor = k.Input(shape = (1,), name = 'scale_factor')

In [9]:
# 2nd Level models
demand = k.LSTM(units = 128, activation = 'tanh')(input_demand)

store = k.Embedding(input_dim = (int(data['store_number'].max() + 1)), output_dim = 25)(input_store)
store = k.Flatten()(store)

biz_cd = k.Embedding(input_dim=(int(data['biz_cd_int'].max() + 1)), output_dim = 25)(input_biz_cd)
biz_cd = k.Flatten()(biz_cd)

scale_factor = k.RepeatVector(testSize)(input_scale_factor)

In [10]:
# Merged Level
merged_level = k.concatenate([demand, store, biz_cd])
merged_level = k.Dense(256, activation='relu')(merged_level)

In [11]:
# Decoder LSTM with predictions
predictions = k.RepeatVector(testSize)(merged_level)
predictions = k.LSTM(256, return_sequences=True, activation='tanh')(predictions)
predictions = k.TimeDistributed(k.Dense(256, activation = 'relu'))(predictions)
predictions = k.TimeDistributed(k.Dense(2, activation = 'softplus'))(predictions)

In [12]:
# Append the scale factor to the end to be used in loss function for training
output = k.concatenate([predictions, scale_factor])
model = tf.keras.models.Model([input_demand, input_store, input_biz_cd, input_scale_factor], outputs = output)
model.compile(optimizer='adam', loss = custom_loss)

In [43]:
train_gen = custom_data_gen(data=data, batchSize=batchSize, 
                                              probs=data["sampling_probability"].tolist(), 
                                              testSize=testSize, windowSize=windowSize)
test_gen = custom_data_gen(data=data, batchSize=1, 
                                              probs=data["sampling_probability"].tolist(), 
                                              testSize=testSize, windowSize=windowSize)

In [152]:
model.fit_generator(generator=train_gen, steps_per_epoch= data.shape[0] / batchSize, epochs=5)

Epoch 1/5
10048/10048 [==============================] - 1285s 128ms/step - loss: 8372.1150
Epoch 2/5
10048/10048 [==============================] - 1276s 127ms/step - loss: 8358.4048
Epoch 3/5
10048/10048 [==============================] - 1271s 127ms/step - loss: 8370.0959
Epoch 4/5
10048/10048 [==============================] - 1270s 126ms/step - loss: 8364.6100
Epoch 5/5
10048/10048 [==============================] - 1264s 126ms/step - loss: 8376.4751


In [160]:
test = next(test_gen)
pred = model.predict(test[0]) 

In [161]:
check = test[1]
check

array([[[ 9.],
        [11.],
        [ 8.],
        [13.],
        [ 6.],
        [ 8.],
        [ 9.],
        [ 8.],
        [14.],
        [12.],
        [ 6.],
        [18.]]])

In [164]:
mu = pred[:,:,0] * pred[:,:,2]
alpha = (pred[:,:,1] / np.sqrt(pred[:,:,2]))
mu = np.reshape(mu, -1)
alpha = np.reshape(alpha, -1)

In [167]:
outcomes = tfp.distributions.NegativeBinomial(mu, alpha)
sess = tf.Session()
sess.run(outcomes.sample())
mu

array([3.0144993e-09, 5.5233938e-12, 2.1023302e-12, 1.8403772e-12,
       1.8074475e-12, 1.8030471e-12, 1.8024419e-12, 1.8023594e-12,
       1.8023457e-12, 1.8023457e-12, 1.8023457e-12, 1.8023457e-12],
      dtype=float32)

<tfp.distributions.NegativeBinomial 'NegativeBinomial_2/' batch_shape=(3, 12) event_shape=() dtype=float32>

In [38]:
def get_nb_vals(mu, alpha, size):
    """Generate negative binomially distributed samples by
    drawing a sample from a gamma distribution with mean `mu` and
    shape parameter `alpha', then drawing from a Poisson
    distribution whose rate parameter is given by the sampled
    gamma variable."""

    g = stats.gamma.rvs(alpha, scale=mu / alpha, size=size)
    return stats.poisson.rvs(g)

In [65]:
get_nb_vals(25,24,10)

array([25, 13, 20, 27, 27, 32, 26, 18, 41, 27])

In [16]:
        rows = np.random.choice(range(0, data.shape[0]), size=batchSize, p=data["sampling_probability"])
        all_cols = [col for col in data if col.startswith('2')]
        start = np.random.randint(0, len(all_cols) - windowSize - testSize, size=1)
        train_cols = all_cols[int(start):int(start + windowSize)]
        test_cols = all_cols[int(start + windowSize):int(start + windowSize + testSize)]
        
        # Store number and subclass will be passed to embedding layers
        stores_train = data.iloc[rows,]["store_number"].tolist()
        biz_cd_train = data.iloc[rows,]["biz_cd_int"].tolist()
        
        # Scale factor is used to scale training data and will be passed into model
        # to be used to rescale the network output
        scale_factor = data.iloc[rows,]["scale_factor"].tolist()
        
        # We set the max number of classes to 53
        week_train = to_categorical([map(int, [week[-2:] for week in train_cols])], num_classes=53)
        week_train = np.reshape(np.repeat(week_train, batchSize, axis=0), (batchSize, windowSize, 53))

        week_test = to_categorical([map(int, [week[-2:] for week in test_cols])], num_classes=53)
        week_test = np.reshape(np.repeat(week_test, batchSize, axis=0), (batchSize, testSize, 53))
        
        x_data = data.iloc[rows,][train_cols].div(data.iloc[rows,]["scale_factor"], axis = 0).values
        x_data = np.reshape(x_data, (batchSize, windowSize, 1))
        x_data = np.concatenate((x_data, week_train), axis=2)
        
        y_data = data.iloc[rows,][test_cols].values
        y_data = np.reshape(y_data, (batchSize, testSize, 1))